# XGBoost Model

In [23]:
import pandas as pd
from xgboost import XGBRegressor, callback
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
from tqdm import tqdm
from datetime import datetime
import pytz
import json
import joblib 
import os

In [24]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v5.csv")
submission_set = pd.read_csv("./data/submission_set.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v5.csv")

# If necessary change this part to test the model before the training process
df = challenge_set_updated.iloc[:,:]

# Separating features and target variable
X = df.drop('tow', axis=1)
y = df['tow']

In [25]:
# Split the data into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets for early stopping
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Define the best parameters provided
best_params = {
    'subsample': 1.0,
    'reg_lambda': 0.46415888336127775,
    'reg_alpha': 0.166810053720005,
    'min_child_weight': 4,
    'max_depth': 13,
    'learning_rate': 0.01,
    'gamma': 0.4444444444444444,
    'colsample_bytree': 0.6
}

# Initialize the XGBoost model with the provided best parameters
best_model = XGBRegressor(
    **best_params,
    objective='reg:squarederror',
    random_state=42,
    n_estimators=1_000_000,  # Set a high value to allow early stopping to find the best n_estimators
    n_jobs=os.cpu_count() - 1
)

# Define fit parameters including early stopping criteria
fit_params = {
    "eval_metric": "rmse",
    "early_stopping_rounds": 25,
    "verbose": 500
}

# Train the model on the training data with early stopping using the validation set
best_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], **fit_params)

# Update best_params with the best number of estimators found during early stopping
best_params['n_estimators'] = best_model.best_iteration + 1  # +1 because best_iteration is zero-indexed

# Evaluate the final model on the test set
y_pred = best_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Best Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")
print(f"Updated best_params: {best_params}")

[0]	validation_0-rmse:52902.32534


/opt/conda/lib/python3.9/site-packages/xgboost-2.0.3-py3.9-linux-x86_64.egg/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/xgboost-2.0.3-py3.9-linux-x86_64.egg/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[500]	validation_0-rmse:3757.27872
[1000]	validation_0-rmse:3512.21341
[1500]	validation_0-rmse:3437.89495
[2000]	validation_0-rmse:3402.88631
[2500]	validation_0-rmse:3380.16125
[3000]	validation_0-rmse:3363.48018
[3500]	validation_0-rmse:3349.76491
[4000]	validation_0-rmse:3340.10849
[4500]	validation_0-rmse:3332.77265
[5000]	validation_0-rmse:3326.64030
[5500]	validation_0-rmse:3321.61607
[6000]	validation_0-rmse:3317.79789
[6500]	validation_0-rmse:3314.72427
[7000]	validation_0-rmse:3311.92309
[7500]	validation_0-rmse:3309.34501
[8000]	validation_0-rmse:3307.42471
[8500]	validation_0-rmse:3306.09161
[8672]	validation_0-rmse:3305.67471
Best Model Performance - R^2 Score: 0.9962, RMSE: 3249.5795
Updated best_params: {'subsample': 1.0, 'reg_lambda': 0.46415888336127775, 'reg_alpha': 0.166810053720005, 'min_child_weight': 4, 'max_depth': 13, 'learning_rate': 0.01, 'gamma': 0.4444444444444444, 'colsample_bytree': 0.6, 'n_estimators': 8648}


In [26]:
# Save R², RMSE, and hyperparameters
results = {
    'R2': float(r2),
    'RMSE': float(rmse),
    'Best Parameters': {key: (int(value) if isinstance(value, np.integer) else float(value)
                              if isinstance(value, np.floating) else value)
                        for key, value in best_params.items()}
}

# Set timezone to São Paulo (UTC-3)
saopaulo_tz = pytz.timezone('America/Sao_Paulo')
timestamp = datetime.now(saopaulo_tz).strftime('%Y%m%d_%H%M%S')

# Define logs directory, and create them if they don't exist
logs_dir = 'logs'
os.makedirs(logs_dir, exist_ok=True)

# Define file paths within the respective directories
results_file = os.path.join(logs_dir, f'model_results_{timestamp}.txt')

# Save the results to a TXT file
with open(results_file, 'w') as file:
    file.write(f"R2: {results['R2']}\n")
    file.write(f"RMSE: {results['RMSE']}\n")
    file.write("Best Parameters:\n")
    for param, value in results['Best Parameters'].items():
        file.write(f"  {param}: {value}\n")

print(f"Results saved to {results_file}")

Results saved to logs/model_results_20240909_170311.txt


In [27]:
# Display evaluation metrics
print(f"Final Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")

Final Model Performance - R^2 Score: 0.9962, RMSE: 3249.5795


In [28]:
# Define models directory, and create them if they don't exist
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True)

# Train the final model using the full training+validation+test set with the optimal n_estimators
final_model = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42, n_jobs=os.cpu_count() - 1)

# Train the model on the entire training+validation+set data
final_model.fit(X, y, verbose=True)

print("Final model trained successfully using all available data.")

Final model trained successfully using all available data.


In [29]:
# Define file paths within the respective directories
model_file = os.path.join(models_dir, f'trained_model_{timestamp}.joblib')

# Save the trained model to a file in the models folder
joblib.dump(final_model, model_file)
print(f"Model saved to {model_file}")

Model saved to models/trained_model_20240909_170311.joblib


In [30]:
# Use the final model to predict the `tow` for the submission_set_updated
submission_set_features = submission_set_updated.iloc[:,:-1]
submission_set['tow'] = final_model.predict(submission_set_features)

submission_set

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,arrival_time,aircraft_type,wtc,airline,flight_duration,taxiout_time,flown_distance,tow
0,248753821,2022-01-01,3b3de0f3ad0ee192513995c02f7bf7cf,LTFJ,Istanbul Sabiha Gokcen,TR,LFLL,Lyon,FR,2022-01-01T09:44:00Z,2022-01-01T12:48:33Z,B738,M,6351ec1b849adacc0cbb3b1313d8d39b,170,15,1122,70337.132812
1,248753822,2022-01-01,e06dd03d4a879ca37d9e18c1bd7cad16,EBBR,Brussels,BE,KJFK,New York JFK,US,2022-01-01T09:45:00Z,2022-01-01T17:49:51Z,A333,H,bdeeef3a675587d530de70a25d7118d2,470,15,3205,212159.890625
2,248754498,2022-01-01,2d3b1c962c78c4ebeef11bcd51b9e94c,KMIA,Miami,US,EGLL,London Heathrow,GB,2022-01-01T01:52:00Z,2022-01-01T09:55:16Z,B77W,H,5543e4dc327359ffaf5b9c0e6faaf0e1,473,10,3965,223126.703125
3,248757623,2022-01-01,81564432d3ee97c4bdf4cd8f006753dc,EGCN,Doncaster Sheffield,GB,LEAL,Alicante,ES,2022-01-01T08:20:00Z,2022-01-01T11:06:08Z,B38M,M,3922524069809ac4326134429751e26f,156,10,986,62615.425781
4,248763603,2022-01-01,84be079d7e660db105d91f600b4b3d59,EIDW,Dublin,IE,LFLL,Lyon,FR,2022-01-01T11:01:00Z,2022-01-01T13:00:43Z,A320,M,a73f82288988b79be490c6322f4c32ed,105,15,686,63239.503906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105954,258066302,2022-12-31,2d3b4446c4d05a25196a9d52cab936fb,LTFJ,Istanbul Sabiha Gokcen,TR,EKCH,Copenhagen,DK,2022-12-31T09:36:00Z,2022-12-31T13:12:17Z,B38M,M,6351ec1b849adacc0cbb3b1313d8d39b,201,15,1199,67519.648438
105955,258068609,2022-12-31,253fd692ed441fac523081471c067772,LOWW,Vienna,AT,KIAD,Washington Dulles,US,2022-12-31T09:49:00Z,2022-12-31T19:38:26Z,B763,H,5d407cb11cc29578cc3e292e743f5393,575,14,3937,174218.156250
105956,258068876,2022-12-31,c9fca302ca2e28acab0eb0bb1b46f11b,LTFM,iGA Istanbul,TR,LSZH,Zurich,CH,2022-12-31T09:25:00Z,2022-12-31T12:24:24Z,A321,M,6351ec1b849adacc0cbb3b1313d8d39b,154,25,988,75951.953125
105957,258064675,2022-12-31,00f96ad0e382476649574ba044c764fc,EHAM,Amsterdam,NL,EDDF,Frankfurt,DE,2022-12-31T10:04:21Z,2022-12-31T10:55:35Z,A320,M,f502877cab405652cf0dd70c2213e730,42,9,240,61543.136719


In [31]:
submission_set['tow'].describe()

count    105959.000000
mean      79420.414062
std       53113.781250
min       16817.580078
25%       56681.082031
50%       63639.144531
75%       73300.988281
max      345793.531250
Name: tow, dtype: float64

In [32]:
challenge_set_updated['tow'].describe()

count    369013.000000
mean      79482.257229
std       53250.919631
min       14944.000000
25%       55836.000000
50%       63852.000000
75%       73756.000000
max      351327.000000
Name: tow, dtype: float64

In [33]:
# Define the submissions directory and create it if it doesn't exist
submissions_dir = 'submissions'
os.makedirs(submissions_dir, exist_ok=True)

# Save the submission with a timestamp in the filename
submission_file = os.path.join(submissions_dir, f"submission_{timestamp}.csv")
submission_set.to_csv(submission_file, index=False)